In [0]:

# STEP 1: Connection Details
storage_account = "staanildatalake01"
# Using the brand new key you just provided
access_key = "UQcUrzR7eJ8Nj4Twb60HsWG5EnVg3noLbY9dvK7mdaIvXQk3u1blDJzKAm3FeC+Qh6GfGZwLTBua+ASt4HG1dw=="

# STEP 2: Configure Authentication
spark.conf.set(f"fs.azure.account.key.{storage_account}.dfs.core.windows.net", access_key)

# STEP 3: Point to your file (.txt format as seen in your screenshot)
# Container is 'bronze'
file_path = f"abfss://bronze@{storage_account}.dfs.core.windows.net/data_de80c0b1-21e0-426e-b5c1-75cdd12a61e2_21b68a3a-c9d3-49cb-bd92-6c0716a535c4.txt"

# STEP 4: Read and Display
try:
    # Reading as text because the file has a .txt extension in your portal
    df = spark.read.text(file_path)
    display(df)
    print("Anil, Success! The data is now visible below.")
except Exception as e:
    print(f"Error: {e}")


value
"SalesID,CustomerName,ProductName,Amount,ModifiedDate"
"1,""Anil Kumar"",""Laptop"",75000.00,2026-01-05 09:00:00.0000000"
"2,""Suresh Reddy"",""Monitor"",15000.00,2026-01-10 10:30:00.0000000"
"3,""Priya Sharma"",""Keyboard"",2500.00,2026-01-15 11:45:00.0000000"
"4,""Ravi Teja"",""Mouse"",1200.00,2026-01-20 14:20:00.0000000"
"5,""Sneha Lata"",""Printer"",12000.00,2026-01-25 16:10:00.0000000"
"6,""Vicky Kaushal"",""Smartphone"",45000.00,2026-02-01 08:30:00.0000000"
"7,""Rahul Dravid"",""Headphones"",3500.00,2026-02-05 12:00:00.0000000"
"8,""Mahesh Babu"",""Smartwatch"",8000.00,2026-02-08 15:50:00.0000000"
"9,""Allu Arjun"",""Tablet"",30000.00,2026-02-12 17:05:00.0000000"


Anil, Success! The data is now visible below.


In [0]:
# STEP 1: Configure access to ADLS using the Access Key you provided
storage_account = "staanildatalake01"
container = "bronze"
# This is the Access Key provided by you
storage_key = "UQcUrzR7eJ8Nj4Twb60HsWG5EnVg3noLbY9dvK7mdaIvXQk3u1blDJzKAm3FeC+Qh6GfGZwLTBua+ASt4HG1dw==" 

# Configuring Spark to use the Access Key directly to avoid "Access Denied" errors
spark.conf.set(
    f"fs.azure.account.key.{storage_account}.dfs.core.windows.net",
    storage_key
)

# STEP 2: Define the file path using ABFSS (Secure Access)
# Using wildcard (*) to pick any file starting with 'data_' in the bronze container
file_path = f"abfss://{container}@{storage_account}.dfs.core.windows.net/data_*"

# Reading the raw data from the Bronze layer
df = spark.read.text(file_path)

# STEP 3: Import required PySpark functions
from pyspark.sql.functions import split, col

# STEP 4: Filter out the header row to avoid data type errors
# This removes the row containing column names like 'SalesID'
df_raw = df.filter(~col("value").contains("SalesID"))

# STEP 5: Split the single string column into multiple structured columns
# Casting 'SalesID' to Integer and 'Amount' to Double for the Silver layer
df_silver = df_raw.select(
    split(col("value"), ",").getItem(0).cast("int").alias("SalesID"),
    split(col("value"), ",").getItem(1).alias("CustomerName"),
    split(col("value"), ",").getItem(2).alias("ProductName"),
    split(col("value"), ",").getItem(3).cast("double").alias("Amount"),
    split(col("value"), ",").getItem(4).alias("ModifiedDate")
)

# STEP 6: Display the cleaned and structured Silver table
# This command is essential to see your data table in Databricks
display(df_silver)

# Success message
print("Anil, Success! The Silver Layer transformation is complete and visible below.")

SalesID,CustomerName,ProductName,Amount,ModifiedDate
1,"""Anil Kumar""","""Laptop""",75000.0,2026-01-05 09:00:00.0000000
2,"""Suresh Reddy""","""Monitor""",15000.0,2026-01-10 10:30:00.0000000
3,"""Priya Sharma""","""Keyboard""",2500.0,2026-01-15 11:45:00.0000000
4,"""Ravi Teja""","""Mouse""",1200.0,2026-01-20 14:20:00.0000000
5,"""Sneha Lata""","""Printer""",12000.0,2026-01-25 16:10:00.0000000
6,"""Vicky Kaushal""","""Smartphone""",45000.0,2026-02-01 08:30:00.0000000
7,"""Rahul Dravid""","""Headphones""",3500.0,2026-02-05 12:00:00.0000000
8,"""Mahesh Babu""","""Smartwatch""",8000.0,2026-02-08 15:50:00.0000000
9,"""Allu Arjun""","""Tablet""",30000.0,2026-02-12 17:05:00.0000000
10,"""Ram Charan""","""Hard Drive""",6000.0,2026-02-14 10:00:00.0000000


Anil, Success! The Silver Layer transformation is complete and visible below.


In [0]:
# CODE 3: Save Data to Silver Layer

# STEP 1: Define the Silver path and storage details
# We ensure the storage account name is present
storage_account = "staanildatalake01"
silver_output_path = f"abfss://silver@{storage_account}.dfs.core.windows.net/sales_cleaned_data"

# STEP 2: Write the clean data as a Parquet file
# Using 'overwrite' mode so it refreshes the data every time you run the notebook
try:
    df_silver.write.mode("overwrite").parquet(silver_output_path)
    print("Success! Anil, your cleaned data is now safely stored in the Silver Layer.")
except Exception as e:
    print(f"Error saving to Silver Layer: {e}")

Success! Anil, your cleaned data is now safely stored in the Silver Layer.


In [0]:
# STEP 1: Import aggregation functions
from pyspark.sql.functions import sum, desc, count

# STEP 2: Business Logic - Product wise total sales
# We want to know which product earned how much money
df_gold = df_silver.groupBy("ProductName") \
    .agg(sum("Amount").alias("TotalRevenue"), 
         count("SalesID").alias("TotalOrders")) \
    .orderBy(desc("TotalRevenue"))

# STEP 3: Display the Final Gold Insights
print("Anil, here is your Business Report (Gold Layer):")
display(df_gold)

Anil, here is your Business Report (Gold Layer):


ProductName,TotalRevenue,TotalOrders
"""Laptop""",75000.0,1
"""Graphics Card""",55000.0,1
"""Smartphone""",45000.0,1
"""Tablet""",30000.0,1
"""Monitor""",15000.0,1
"""Printer""",12000.0,1
"""Smartwatch""",8000.0,1
"""Hard Drive""",6000.0,1
"""Webcam""",4000.0,1
"""Headphones""",3500.0,1


In [0]:
# STEP 1: Define Gold Output Path
gold_path = f"abfss://gold@{storage_account}.dfs.core.windows.net/sales_final_report"

# STEP 2: Save the Business Insights as a Parquet file
df_gold.write.mode("overwrite").parquet(gold_path)

print("Congratulations Anil! Project 1 is fully completed and saved in the Gold Layer.")

Congratulations Anil! Project 1 is fully completed and saved in the Gold Layer.


In [0]:
# CODE to check Silver Layer Data
# STEP 1: Define the path where you saved the silver data
storage_account = "staanildatalake01"
silver_path = f"abfss://silver@{storage_account}.dfs.core.windows.net/sales_cleaned_data"

# STEP 2: Read the Parquet file back into a DataFrame
# Because it is a parquet file, we use spark.read.parquet()
df_check = spark.read.parquet(silver_path)

# STEP 3: Display the data
# This will show you the clean table with all columns properly
display(df_check)

SalesID,CustomerName,ProductName,Amount,ModifiedDate
1,"""Anil Kumar""","""Laptop""",75000.0,2026-01-05 09:00:00.0000000
2,"""Suresh Reddy""","""Monitor""",15000.0,2026-01-10 10:30:00.0000000
3,"""Priya Sharma""","""Keyboard""",2500.0,2026-01-15 11:45:00.0000000
4,"""Ravi Teja""","""Mouse""",1200.0,2026-01-20 14:20:00.0000000
5,"""Sneha Lata""","""Printer""",12000.0,2026-01-25 16:10:00.0000000
6,"""Vicky Kaushal""","""Smartphone""",45000.0,2026-02-01 08:30:00.0000000
7,"""Rahul Dravid""","""Headphones""",3500.0,2026-02-05 12:00:00.0000000
8,"""Mahesh Babu""","""Smartwatch""",8000.0,2026-02-08 15:50:00.0000000
9,"""Allu Arjun""","""Tablet""",30000.0,2026-02-12 17:05:00.0000000
10,"""Ram Charan""","""Hard Drive""",6000.0,2026-02-14 10:00:00.0000000
